<div>
首发于<a href="https://www.heywhale.com/mw/project/662ddcb509492666bdf6bbc1"><img src="https://open-cdn.kesci.com/admin/scmzi3wk6/Open%20in%20ModelWhale.svg" alt="Open In ModelWhale" style="height:23px;"></a>

可按此键一键运行⬆️
</div>

## 用深度学习补全观测数据中的缺测数据  

在现实生活中，时间序列数据就像一部连续播放的电影，记录着各种事物随时间变化的精彩瞬间。特别是在气象学等地球科学领域，多变量（`Multivariate`）时间序列的应用无处不在，它帮助我们揭示气候、环境和生态系统背后的规律。  

想象一下，每一次温度、湿度、风速的测量，都是这部“地球电影”中不可或缺的一帧画面。然而，现实生活并非总是完美无瑕，就像拍摄电影时偶尔会出现镜头故障一样，在实际采集时间序列数据的过程中，也常常会遇到传感器失效、传输错误等问题，导致部分数据缺失。这些缺失值就像是电影中的空白片段，不仅让整部“电影”的连贯性受到影响，降低了数据的可读性和故事性，同时也给科学家们开展高级分析和模式识别任务带来了巨大挑战，比如对气候变化进行分类或对不同区域天气现象进行聚类。  

因此，在深入挖掘这些珍贵的时间序列信息之前，我们必须先解决一个关键问题：如何巧妙地处理那些“丢失的画面”？这就需要运用一系列创新的数据预处理方法，来填补缺失值，确保我们的“地球电影”能够完整流畅地讲述其背后的故事。

### 传统的数据缺测处理方法：数据删除和数据差补  
在处理时间序列数据时，我们经常会遇到一个问题：数据缺失。这就像是我们在观看一部精彩的电影时，突然发现有些画面不见了，这让整个故事变得不连贯，也让我们难以完全理解电影想要传达的信息。在科学研究中，这些缺失的数据点可能会影响我们对气象变化、环境状况等问题的深入分析。  

那么，我们该如何处理这些缺失的数据呢？传统的做法主要有两种：一是删除法，简单来说，就是把这些“丢失的画面”从电影中剪掉。这种方法虽然直接，但可能会导致我们的电影变得支离破碎，丢失重要的信息，甚至可能因为删除了关键帧而改变了整个故事的主线  
另一种方法是数据插补，这就好比是我们根据电影的上下文，来推测那些丢失的画面应该是什么，然后用我们的想象来填补这些空白。这种方法的优势在于，我们能够保留电影的原貌，不会因为删除画面而引入新的偏差。而且，即使是一些不完整的画面，也可能包含着对我们理解故事至关重要的线索。  

但是，数据插补也有其难点，那就是我们如何确定应该填补什么样的内容。这里就需要借助统计学和机器学习的技术了。比如，我们可以使用线性回归来预测缺失的数值，或者用平均值、中位数来填补，甚至可以通过寻找相似的“电影片段”来估计那些丢失的画面。不过，这些方法通常需要对数据的分布做出一些假设，如果这些假设不正确，那么我们的填补就可能带有偏差，影响我们对电影真实情节的理解。  

因此，在处理时间序列数据中的缺失值时，我们需要谨慎地选择方法，并不断地验证和调整我们的假设，以确保我们能够尽可能地还原数据的真实面貌，让我们的“地球电影”能够流畅地播放，讲述出准确而完整的故事。

#### 数据集：北京多站点空气质量数据集  
本项目中使用北京多站点空气质量数据集为例。该数据集来自`Zhang S, Guo B, Dong A, et al. Cautionary tales on air-quality improvement in Beijing[J]. Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences, 2017, 473(2205): 20170457.`  
可以点击[这里](https://www.heywhale.com/org/meteoda/dataset/65dff755d0681f06ab9bfb23)下载。  
先简单打开它看看情况。

In [1]:
import pandas as pd
df_origin = pd.read_csv("/home/mw/input/bmaq3047/beijing_multisite_air_quality.csv",index_col=0)
df_origin.head()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
No,,,,,,,,,,,,,,,,,
1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


In [2]:
# 但是数据太大了。。。此次案例我们只选用2013年3月~4月的数据
df_origin = df_origin[(df_origin['year'] == 2013) & ((df_origin['month'] == 3) | (df_origin['month'] == 4))]
df_origin

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
No,,,,,,,,,,,,,,,,,
1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,2013,4,30,19,6.0,135.0,3.0,26.0,400.0,78.0,20.9,1005.9,4.4,0.0,SE,0.5,Wanshouxigong
1461,2013,4,30,20,45.0,169.0,9.0,61.0,700.0,62.0,22.3,1007.0,-1.8,0.0,NE,1.6,Wanshouxigong
1462,2013,4,30,21,50.0,124.0,9.0,98.0,900.0,20.0,21.0,1007.6,-7.6,0.0,E,2.4,Wanshouxigong


In [3]:
# 统计整个表格，可以看出所有变量（除了时间和站点名）都存在缺测值
df_origin.describe()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,17568.0,17568.000000,17568.000000,17568.000000,17457.000000,17474.000000,17425.000000,17038.000000,16515.000000,17208.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000
mean,2013.0,3.491803,15.754098,11.500000,83.887151,112.707136,29.780521,57.350609,1182.679685,59.116730,9.133794,1010.054263,-4.971397,0.019109,2.106199
std,0.0,0.499947,8.807207,6.922384,79.091267,90.381114,29.947173,38.008304,934.329961,36.539751,6.138903,7.519147,6.556816,0.158649,1.502764
min,2013.0,3.000000,1.000000,0.000000,2.000000,2.000000,0.285600,1.642400,100.000000,0.214200,-7.200000,988.500000,-25.500000,0.000000,0.000000
25%,2013.0,3.000000,8.000000,5.750000,18.000000,42.000000,7.000000,28.000000,500.000000,28.000000,4.800000,1004.800000,-9.800000,0.000000,1.100000
50%,2013.0,3.000000,16.000000,11.500000,64.000000,99.000000,19.000000,53.000000,1000.000000,63.000000,8.600000,1009.700000,-4.100000,0.000000,1.700000
75%,2013.0,4.000000,23.000000,17.250000,120.000000,156.000000,44.000000,79.000000,1600.000000,87.000000,12.900000,1015.400000,-0.200000,0.000000,2.800000
max,2013.0,4.000000,31.000000,23.000000,558.000000,987.000000,198.000000,273.000000,10000.000000,674.000000,29.400000,1033.700000,11.300000,5.700000,13.200000


In [4]:
# 一共有12个站点
num_samples = len(df_origin['station'].unique())
num_samples

12

In [5]:
# 验证一下，发现每个站点都是2013年3月1日0时~2013年4月30日23时， 长度一致 (1464个时次)
from datetime import datetime,timedelta
print(len(df_origin)/num_samples)
print((datetime(2013,4,30,23)-datetime(2013,3,1,0))/timedelta(hours=1) + 1 )

1464.0
1464.0


In [6]:
# 统计整个表格，可以看出所有变量（除了时间和站点名）都存在缺测值
df_origin.describe()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,17568.0,17568.000000,17568.000000,17568.000000,17457.000000,17474.000000,17425.000000,17038.000000,16515.000000,17208.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000
mean,2013.0,3.491803,15.754098,11.500000,83.887151,112.707136,29.780521,57.350609,1182.679685,59.116730,9.133794,1010.054263,-4.971397,0.019109,2.106199
std,0.0,0.499947,8.807207,6.922384,79.091267,90.381114,29.947173,38.008304,934.329961,36.539751,6.138903,7.519147,6.556816,0.158649,1.502764
min,2013.0,3.000000,1.000000,0.000000,2.000000,2.000000,0.285600,1.642400,100.000000,0.214200,-7.200000,988.500000,-25.500000,0.000000,0.000000
25%,2013.0,3.000000,8.000000,5.750000,18.000000,42.000000,7.000000,28.000000,500.000000,28.000000,4.800000,1004.800000,-9.800000,0.000000,1.100000
50%,2013.0,3.000000,16.000000,11.500000,64.000000,99.000000,19.000000,53.000000,1000.000000,63.000000,8.600000,1009.700000,-4.100000,0.000000,1.700000
75%,2013.0,4.000000,23.000000,17.250000,120.000000,156.000000,44.000000,79.000000,1600.000000,87.000000,12.900000,1015.400000,-0.200000,0.000000,2.800000
max,2013.0,4.000000,31.000000,23.000000,558.000000,987.000000,198.000000,273.000000,10000.000000,674.000000,29.400000,1033.700000,11.300000,5.700000,13.200000


#### 使用传统方法进行数据删除  
一般来说，面对缺测数据，要对其预处理，才能进行下游任务。最简单的方法当然是把含有缺测数据的行直接删除了。

In [7]:
# 需要删除含有缺测数据的行。这里使用pandas的dropna方法。
# 删除含有任何NaN值的行
df_cleaned = df_origin.dropna(inplace=False)

# 查看清理后的数据
df_cleaned.describe()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,15814.0,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000,15814.000000
mean,2013.0,3.492412,15.569432,11.665107,85.639433,114.368123,30.257502,57.521335,1200.436006,58.707941,9.161787,1010.013052,-4.809795,0.020128,2.080056
std,0.0,0.499958,8.792381,6.929264,79.827174,91.133735,30.152618,38.161482,938.111425,36.583920,6.064021,7.545932,6.536386,0.163408,1.478850
min,2013.0,3.000000,1.000000,0.000000,2.000000,2.000000,0.571200,2.000000,100.000000,0.214200,-7.200000,988.900000,-25.500000,0.000000,0.000000
25%,2013.0,3.000000,8.000000,6.000000,19.000000,43.000000,7.000000,28.000000,500.000000,27.000000,4.900000,1004.700000,-9.600000,0.000000,1.000000
50%,2013.0,3.000000,15.000000,12.000000,65.000000,100.000000,19.000000,53.000000,1000.000000,62.000000,8.600000,1009.600000,-3.800000,0.000000,1.700000
75%,2013.0,4.000000,23.000000,18.000000,122.000000,159.000000,44.000000,80.000000,1600.000000,87.000000,12.800000,1015.400000,-0.100000,0.000000,2.800000
max,2013.0,4.000000,31.000000,23.000000,558.000000,987.000000,198.000000,273.000000,10000.000000,674.000000,29.400000,1033.700000,11.300000,5.700000,13.200000


这种方法问题很大。因为在删除了包含缺测数据的行的同时，我们也把这个行里的其他数据，及其包含的有效信息给删除了。这是我们不想要的。

#### 基于线性插值等方案进行数据插补  
如果我们不想直接删除缺测数据及其所在行，我们就会想办法进行数据的填充。  

对于数值类型的数据，我们常会用线性方法进行插补; 而如果是非数据类型的数据，最常见的方法是用最频繁的值进行插补。

In [8]:
# 使用线性插值方法对数值型数据进行插补
# 对于非数值型数据，使用最高频数据进行插补

# 导入必要的库
import pandas as pd
import numpy as np  # 由于使用了np.number，需要导入numpy库

def impute_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    对给定的DataFrame进行数据插补。

    对于数值型数据，使用线性插值方法进行插补；
    对于非数值型数据，使用最高频数据进行插补。

    参数:
        df (pd.DataFrame): 需要插补的DataFrame。

    返回:
        pd.DataFrame: 插补后的DataFrame。

    示例:
        >>> df = pd.DataFrame({'A': [1, np.nan, 3], 'B': ['a', 'b', np.nan]})
        >>> impute_data(df)
           A  B
        0  1.0  a
        1  2.0  b
        2  3.0  b
    """
    x = df.copy()
    # 对数值型数据进行线性插补
    df_numeric = x.select_dtypes(include=[np.number])
    x[df_numeric.columns] = df_numeric.interpolate()

    # 对非数值型数据进行最高频数据插补
    df_non_numeric = x.select_dtypes(exclude=[np.number])
    for column in df_non_numeric.columns:
        # 找到最高频的数据
        mode_value: str = x[column].mode()[0]
        # 对缺失值进行插补，避免使用inplace=True以符合pandas即将到来的更新
        x[column] = x[column].fillna(mode_value)

    return x

# 对原始数据进行插补
df_imputed = impute_data(df_origin)

# 查看插补后的数据
df_imputed.describe()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,17568.0,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000,17568.000000
mean,2013.0,3.491803,15.754098,11.500000,83.735627,113.369251,29.754473,56.886257,1176.241149,59.132991,9.133794,1010.054263,-4.971397,0.019109,2.106199
std,0.0,0.499947,8.807207,6.922384,78.968947,92.542279,29.898506,38.348979,927.412191,38.584425,6.138903,7.519147,6.556816,0.158649,1.502764
min,2013.0,3.000000,1.000000,0.000000,2.000000,2.000000,0.285600,1.642400,100.000000,0.214200,-7.200000,988.500000,-25.500000,0.000000,0.000000
25%,2013.0,3.000000,8.000000,5.750000,18.000000,41.000000,7.000000,27.000000,500.000000,27.000000,4.800000,1004.800000,-9.800000,0.000000,1.100000
50%,2013.0,3.000000,16.000000,11.500000,63.000000,99.000000,19.000000,52.000000,1000.000000,62.000000,8.600000,1009.700000,-4.100000,0.000000,1.700000
75%,2013.0,4.000000,23.000000,17.250000,120.000000,157.000000,44.000000,79.000000,1600.000000,87.000000,12.900000,1015.400000,-0.200000,0.000000,2.800000
max,2013.0,4.000000,31.000000,23.000000,558.000000,987.000000,198.000000,273.000000,10000.000000,674.000000,29.400000,1033.700000,11.300000,5.700000,13.200000


In [9]:
# 对其进行可视化
import matplotlib.pyplot as plt
import pandas as pd

# 创建时间序列作为横轴
时间序列 = pd.date_range(start="2013-03-01", end="2013-04-30 23:00:00", freq='H')

plt.figure(figsize=(20, 6))
# 仅绘制前744个数据点
plt.plot(时间序列, df_imputed['CO'][:1464], label='imputed CO')
plt.plot(时间序列, df_origin['CO'][:1464], label='origin CO')
plt.legend()
plt.xlabel('Time')
plt.ylabel('CO')
plt.title('201303~201304 CO origin vs imputed')
plt.xticks(rotation=45)  # 旋转x轴标签，以便更清晰地显示
plt.show()

<Figure size 2000x600 with 1 Axes>

线性插值方法对数值型数据进行插补时，主要基于相邻数据点之间的线性关系来估计缺失值。这种方法简单且易于实现，但在某些情况下可能会带来以下坏处：  
1. 不适用于非线性数据：如果数据集中的变量之间的关系是非线性的，线性插值可能无法准确反映这种复杂的关系，从而导致插补的数据与实际情况有较大偏差。  
2. 对异常值敏感：线性插值依赖于相邻的数据点。如果相邻的数据点包含异常值，插补的结果可能会受到影响，导致插补值不准确。  
3. 时间序列的周期性和趋势：对于时间序列数据，线性插值可能无法很好地处理数据的周期性和趋势变化。例如，在气象数据中，温度和湿度等变量往往表现出明显的季节性变化，简单的线性插补可能无法捕捉到这种周期性模式。  
4. 数据的结构性缺失：如果数据缺失不是随机发生的，而是由于某些未观测到的变量引起的结构性缺失，线性插补可能无法恰当地反映数据的真实分布，从而影响下游应用的分析和预测准确性。  
5. 过度平滑：线性插补可能导致数据过度平滑，尤其是在数据变化较为剧烈的区域。这种过度平滑可能掩盖数据中的重要特征和模式，降低模型学习的效率和准确性。  
因此，在选择数据插补方法时，需要根据数据的特性和下游任务的需求，综合考虑使用最合适的插补方法。对于复杂或非线性的数据关系，可能需要考虑更高级的插补方法，如基于机器学习的插补方法（例如K最近邻插补、决策树插补、深度学习模型等），这些方法能够更好地捕捉数据之间的复杂关系，提高插补的准确性和可靠性。

### SATIS: 基于自我注意力的时间序列补全  

在当今的数据处理领域，自我注意力机制已经得到了广泛的应用，但在时间序列补全方面的应用还相对有限。过去，时间序列补全的最优模型大多基于循环神经网络（RNN）。其中许多采用的是自回归模型，这种模型很容易受到累积误差的影响。另一些虽然不是自回归的，但其提出的多分辨率补全算法包含循环结构，这大大降低了补全速度。  

自注意力机制则不同，它既不是自回归的，又能克服RNN在速度和记忆上的限制，避免了累积误差的问题，这对于提高补全质量和速度都大有裨益。因此，[Wenjie Du](https://github.com/WenjieDu) 等提出了一种名为`SAITS`（基于自我注意力的时间序列补全）的新型模型，通过一种联合优化的训练方法来学习缺失值，这种方法同时考虑了补全和重建的任务。

#### 联合优化训练方法（Joint-optimization Training Approach）  

在时间序列数据的处理中，我们经常会遇到数据缺失的情况。为了解决这个问题，SATIS引入了两种任务：掩蔽预测任务（MIT）和观测重建任务（ORT）。这两种任务相互补充，共同帮助我们更准确地补全缺失的数据。  

![Image Name](https://cdn.kesci.com/upload/s9ln9efcs8.png?imageView2/0/w/960/h/960)

##### 掩蔽预测任务（MIT）  
掩蔽预测任务（MIT）有点像是我们和模型玩的一个“猜猜看”游戏。我们随机选取一部分观测到的数据，将其“掩蔽”起来，也就是不让模型看到这些数据。然后，我们让模型尝试去预测这些被掩蔽的数据应该是多少。这样做的目的是迫使模型学会如何准确预测缺失的值。我们通过计算模型预测值和真实值之间的平均绝对误差（MAE）来衡量模型的预测效果。

##### 观测重建任务（ORT）  
这个任务相对简单，就是让模型尽量准确地重建那些没有被掩蔽的观测数据。我们同样使用MAE来衡量模型重建的效果。这个任务的重要性在于，它不仅要求模型能够预测缺失的数据，还要求模型能够保持观测数据的准确性。

#### SAITS模型  
![Image Name](https://ar5iv.labs.arxiv.org/html/2202.08516/assets/x4.png?imageView2/0/w/960/h/960)  
我们设计的SAITS模型由两个加权对角掩蔽自我注意力（DMSA）块组成，这使得SAITS摆脱了RNN的束缚，能够显式地捕捉时间步之间的时序依赖性和特征相关性。

#### 使用SATIS对北京多站点空气质量数据集进行缺测补全

In [10]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pypots.imputation import SAITS  # version==0.3.1
from pypots.utils.metrics import calc_mae

In [11]:
# 数据预处理
X = df_origin

In [12]:
num_samples = len(X['station'].unique())
num_steps = round(len(X)/12)
X = X.drop(['year', 'month', 'day', 'hour', 'wd', 'station'], axis=1)
num_features = len(X.columns)

In [13]:
scaler = StandardScaler()
X = scaler.fit_transform(X.to_numpy())
import joblib
joblib.dump(scaler, 'scaler.pkl')
X = X.reshape(num_samples,num_steps, num_features)

In [14]:
dataset = {"X": X}  # 构造satis的模型输入格式

In [15]:
# 使用PyPOTS进行模型训练
saits = SAITS(
    n_steps=num_steps,
    n_features=num_features,
    n_layers=2, d_model=256, d_inner=128, n_heads=4, d_k=64, d_v=64, 
    batch_size = 128,
    epochs=200,
    patience=10,
    saving_path="save_path")
# 这里我使用整个数据集作为训练集，因为模型看不到地面真相，你也可以将其拆分为训练/验证/测试集
saits.fit(dataset)
imputation = saits.predict(dataset)  # 补全原始缺失值和人为缺失值

2024-03-12 07:48:31 [INFO]: No given device, using default device: cuda
2024-03-12 07:48:31 [INFO]: Model files will be saved to save_path/20240312_T074831
2024-03-12 07:48:31 [INFO]: Tensorboard file will be saved to save_path/20240312_T074831/tensorboard
2024-03-12 07:48:31 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 1,350,150
2024-03-12 07:48:33 [INFO]: Epoch 001 - training loss: 1.6468
2024-03-12 07:48:33 [INFO]: Epoch 002 - training loss: 1.5624
2024-03-12 07:48:33 [INFO]: Epoch 003 - training loss: 1.4337
2024-03-12 07:48:33 [INFO]: Epoch 004 - training loss: 1.3377
2024-03-12 07:48:34 [INFO]: Epoch 005 - training loss: 1.2635
2024-03-12 07:48:34 [INFO]: Epoch 006 - training loss: 1.1979
2024-03-12 07:48:34 [INFO]: Epoch 007 - training loss: 1.1590
2024-03-12 07:48:34 [INFO]: Epoch 008 - training loss: 1.1176
2024-03-12 07:48:34 [INFO]: Epoch 009 - training loss: 1.0843
2024-03-12 07:48:34 [INFO]: Epoch 010 - training loss: 1.0619

In [19]:
#把补全后的数据重新reshape回原来的数据，然后用scaler恢复到原样
imputation = imputation["imputation"].reshape(-1, num_features)
imputation = scaler.inverse_transform(imputation)

#### 结果可视化

In [20]:
import matplotlib.pyplot as plt
import pandas as pd

# 创建时间序列作为横轴
时间序列 = pd.date_range(start="2013-03-01", end="2013-04-30 23:00:00", freq='H')

plt.figure(figsize=(20, 6))
# 仅绘制前744个数据点
plt.plot(时间序列, imputation[:1464, 4], label='imputed CO')
plt.plot(时间序列, df_origin['CO'][:1464], label='origin CO')
plt.legend()
plt.xlabel('Time')
plt.ylabel('CO')
plt.title('201303-201304 CO origin VS imputed')
plt.xticks(rotation=45)  # 旋转x轴标签，以便更清晰地显示
plt.show()

<Figure size 2000x600 with 1 Axes>

和上面的线性插值方法对比一下。肉眼可见的效果好~

## 作业：  
上述的补全模型，效果看起来确实不错。但原始数据毕竟还是丢失了。我们没法准确评估到底我们补全的好不好。  
为了更准确、直接地评估模型的性能，尝试这样操作：  
1. 对上面的北京空气质量数据集，进行部分掩码，随机遮盖一部分数据。  
2. 然后再训练模型，补全缺失的数据  
3. 对比被遮盖的原始数据和补全后的数据，从而更直观的感受模型的补全能力。